XLM-Roberta on multilingual text

v2 - setup the process on entire dataset
v3 - added evaluation on shared data (not considering per annotator); plus 256 tokens per text
model saved in iter 3

0. Install and import libraries

In [ ]:
## install libraries

!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 27.9 MB/s 
     |████████████████████████████████| 596 kB 38.9 MB/s 
     |████████████████████████████████| 86 kB 1.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.8 MB/s 


In [1]:
import pandas as pd
import numpy as np
import random
import json
import regex as re

import matplotlib.pyplot as plt


from google.colab import drive
drive.mount('/content/drive')
LIBRARY_PATH = '/content/drive/MyDrive/NLP PROJECT/Finals/'


Mounted at /content/drive


In [4]:
data_path = 'data/processed/train_DA_2.csv'
df = pd.read_csv(LIBRARY_PATH + data_path)

In [12]:
df.head(2)

,Unnamed: 0,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,...,link_id1,link_id2,text1,title1,meta_keywords1,meta_description1,text2,title2,meta_keywords2,meta_description2
0,0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.0,4.0,...,1484084337,1484110209,"MARTINSBURG, W.Va. — A suspected drunken drive...",Virginia man arrested in fatal DUI crash in We...,"['Highway Fatal-DUI-West Virginia', 'Martinsbu...",Police in West Virginia say a suspected drunke...,"PORT-AU-PRINCE, Haiti — Haitian President Jove...",Haiti's leader marks independence day amid sec...,"['CB-Haiti-Political Turmoil', 'Jean', 'Haiti'...",Haitian President Jovenel Moïse has broken wit...
1,1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.0,4.0,...,1484396422,1483924666,Share This On: Pin 11 Shares (NEWS ROOM GUYANA...,Guyana: Three injured after car crashes into u...,[''],-,BERLIN - A fire at a zoo in western Germany in...,Fire kills more than 30 animals at zoo in west...,"['smg2_world', 'smg_europe', 'smg2_news']",BERLIN - A fire at a zoo in western Germany in...


In [11]:
filtered = df[(df['url1_lang'] != 'en') | (df['url2_lang'] != 'en')]

In [15]:
selected_filter = filtered[['meta_keywords1', 'meta_keywords2', 'meta_description1', 'meta_description2']]

In [ ]:
selected_filter[len]